In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import ensemble

from L5.validation import test_training_indices, cross_validation

In [6]:
from sklearn import datasets
iris = datasets.load_iris()

data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [7]:
iris_labels = data['species']
iris_data = data.drop('species', axis = 1)

In [8]:
random_forest_iris = ensemble.RandomForestClassifier()
random_forest_iris.fit(iris_data, iris_labels)
print(random_forest_iris.score(iris_data, iris_labels))

0.993333333333


In [10]:
print(cross_validation(iris_data, iris_labels,  ensemble.RandomForestClassifier()))

[1.0, 1.0, 1.0, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 0.93333333333333335]


In [11]:
extremly_randomized_trees_iris = ensemble.ExtraTreesClassifier()
extremly_randomized_trees_iris.fit(iris_data, iris_labels)
print(extremly_randomized_trees_iris.score(iris_data, iris_labels))

1.0


In [12]:
print(cross_validation(iris_data, iris_labels, ensemble.ExtraTreesClassifier()))

[1.0, 1.0, 1.0, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 0.93333333333333335]


In [4]:
data = pd.read_csv("L5/data/titanic.csv")
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
data = data.dropna()
data['Sex'] = pd.Categorical(data['Sex']).codes
data['Embarked'] = pd.Categorical(data['Embarked']).codes

In [5]:
titanic_labels = data['Survived']
titanic_data = data.drop('Survived', axis = 1)

In [18]:
random_forest_titanic = ensemble.RandomForestClassifier()
random_forest_titanic.fit(titanic_data, titanic_labels)
print(random_forest_titanic.score(titanic_data, titanic_labels))

0.974719101124


In [19]:
print(cross_validation(titanic_data, titanic_labels, ensemble.RandomForestClassifier()))

[0.80281690140845074, 0.78873239436619713, 0.77464788732394363, 0.78873239436619713, 0.84722222222222221, 0.80281690140845074, 0.76056338028169013, 0.80281690140845074, 0.77464788732394363, 0.86111111111111116]


In [20]:
extremly_randomized_trees_titanic = ensemble.ExtraTreesClassifier()
extremly_randomized_trees_titanic.fit(titanic_data, titanic_labels)
print(extremly_randomized_trees_titanic.score(titanic_data, titanic_labels))

0.98595505618


In [22]:
print(cross_validation(titanic_data, titanic_labels, ensemble.ExtraTreesClassifier()))

[0.71830985915492962, 0.76056338028169013, 0.74647887323943662, 0.78873239436619713, 0.80555555555555558, 0.76056338028169013, 0.71830985915492962, 0.80281690140845074, 0.76056338028169013, 0.83333333333333337]


In [6]:
extremly_randomized_trees_titanic = ensemble.ExtraTreesClassifier(n_estimators=25)
extremly_randomized_trees_titanic.fit(titanic_data, titanic_labels)
print(extremly_randomized_trees_titanic.score(titanic_data, titanic_labels))

0.98595505618


In [7]:
random_forest_titanic = ensemble.RandomForestClassifier(n_estimators=25)
random_forest_titanic.fit(titanic_data, titanic_labels)
print(random_forest_titanic.score(titanic_data, titanic_labels))

0.984550561798


In [8]:
print(cross_validation(titanic_data, titanic_labels, 
                       ensemble.RandomForestClassifier(n_estimators=25)))

[0.76056338028169013, 0.81690140845070425, 0.76056338028169013, 0.78873239436619713, 0.86111111111111116, 0.81690140845070425, 0.74647887323943662, 0.76056338028169013, 0.78873239436619713, 0.875]
